Maestría Data Mining

Data Mining en Economía y Finanzas

Diciembre 2025

Silvana Contreras

# Workflow con zLightGBM
Se entrenó 1 versión de este workflow, cada una con 30 semillas y una semilla primigenia.

Atributos: 1399

Dataset con tratamiento de inflación por IPC, tendencias y ratio_avg de ventana 3 y 6, variables intrames simples (no RF). Sin préstamos_personales, asignación de NA cuando batch completo en 0, asignación de NA en columnas monetarias seleccionadas cuando su correspondiente columna de cantidades es 0, ctrx_quarter_normalizado, kmes, mpayroll_edad, lags y deltas orden 1 y 2. 

Undersampling: 0.1

Hiperparámetros de zLightGBM: 
    -num_iterations: 9999
    -min_data_in_leaf: 20
    -learning_rate: 1.0
    -gradient_bound: 0.1
    -canaritos: 5

Entrenamiento con todos los meses disponibles (se aplica decaimiento exponencial al peso de los meses). 

## Inicializacion

In [1]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-11-21 16:54:34 -03"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,647783,34.6,1431154,76.5,1075209,57.5
Vcells,1193454,9.2,8388608,64.0,1876925,14.4


In [ ]:
PARAM <- list()
PARAM$experimento <- "wfzR3317"
PARAM$semilla_primigenia <- 999199 



In [ ]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

# Preprocesamiento

In [ ]:
Sys.time()
require( "data.table" )

# leo el dataset
dataset_02 <- fread("~/buckets/b1/datasets/competencia_02_crudo.csv.gz" )
dataset_03 <- fread("~/buckets/b1/datasets/competencia_03_crudo.csv.gz" )

#options(timeout = 200)

#dataset_02 <- fread("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/competencia_02_crudo.csv.gz" )
#dataset_03 <- fread("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/competencia_03_crudo.csv.gz" )



[1] "2025-11-21 17:00:49 -03"

In [ ]:

# 6) Unir datasets: opción simple (apilar filas) ---------------------------
#    Esta es la opción natural si 03 tiene los mismos meses + alguno nuevo.

if (setequal(names(dataset_02), names(dataset_03))) {
  
  # ordenamos columnas igual por prolijidad
  setcolorder(dataset_03, names(dataset_02))
  
  dataset_0203 <- rbindlist(list(dataset_02, dataset_03), use.names = TRUE)
  
  cat("\n=== DATASET UNIDO ===\n")
  cat("Filas total: ", nrow(dataset_0203), "\n")
  cat("Rango meses: ", min(dataset_0203$foto_mes), " - ", max(dataset_0203$foto_mes), "\n")
  cat("Conteos por mes:\n")
  print(dataset_0203[, .N, by = foto_mes][order(foto_mes)])
  
} else {
  cat("\nNo se unieron porque las columnas no coinciden. Revisar diferencias listadas arriba.\n")
}



=== DIMENSIONES ===
dataset_02:  4717958 filas x 154 columnas
dataset_03:  165093 filas x 154 columnas

=== PRIMERAS COLUMNAS ===
 [1] "numero_de_cliente"           "foto_mes"                   
 [3] "active_quarter"              "cliente_vip"                
 [5] "internet"                    "cliente_edad"               
 [7] "cliente_antiguedad"          "mrentabilidad"              
 [9] "mrentabilidad_annual"        "mcomisiones"                
[11] "mactivos_margen"             "mpasivos_margen"            
[13] "cproductos"                  "tcuentas"                   
[15] "ccuenta_corriente"           "mcuenta_corriente_adicional"
[17] "mcuenta_corriente"           "ccaja_ahorro"               
[19] "mcaja_ahorro"                "mcaja_ahorro_adicional"     
 [1] "numero_de_cliente"           "foto_mes"                   
 [3] "active_quarter"              "cliente_vip"                
 [5] "internet"                    "cliente_edad"               
 [7] "cliente_antiguedad

In [7]:
dataset <- dataset_0203
dim(dataset)

[1] 4883051     154

In [ ]:
rm(dataset_0203, dataset_02, dataset_03)
gc()


In [ ]:
cols_iniciales <- copy(setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes"
    )
))

### Generacion de la clase_ternaria

In [9]:

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,823950,44.1,1431154,76.5,1431154,76.5
Vcells,1301439734,9929.2,2133886054,16280.3,1454595467,11097.7


[1] "2025-11-21 17:06:18 -03"

In [10]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,645
201901,BAJA+2,729
201901,CONTINUA,122899
201902,BAJA+1,733
201902,BAJA+2,707
201902,CONTINUA,123961
201903,BAJA+1,708
201903,BAJA+2,751
201903,CONTINUA,124508


### clase ternaria con regresos

In [ ]:
# # =============================================================
# # CLASE TERNARIA CON REGRESOS CONSERVADORES  (versión final)
# # -------------------------------------------------------------
# #   - CONTINUA si el cliente vuelve dentro de ≤ 2 meses.
# #   - BAJA+1 si falta f1 y no vuelve (o regresa después de > 2 meses).
# #   - BAJA+2 si tiene f1 consecutivo, falta f2 y no vuelve (o regresa después de > 2 meses).
# #   - Mantiene el criterio de topes finales
# # =============================================================
# 
# 
# DT <- copy(dataset)
# 
# # --- Parámetro: meses máximos de ausencia tolerada como "regreso"
# tol <- 2L
# 
# # --- Índice lineal de mes y orden por cliente
# ds <- DT[, .(
#   pos = .I,
#   numero_de_cliente,
#   foto_mes,
#   periodo0 = (foto_mes %/% 100L)*12L + (foto_mes %% 100L)
# )]
# setorder(ds, numero_de_cliente, periodo0)
# 
# # --- Topes globales (como el código original)
# periodo_ultimo     <- ds[, max(periodo0)]
# periodo_anteultimo <- periodo_ultimo - 1L
# 
# # --- Próximas observaciones del cliente
# ds[, `:=`(
#   periodo1 = shift(periodo0, n=1L, type="lead"),
#   periodo2 = shift(periodo0, n=2L, type="lead")
# ), by = numero_de_cliente]
# 
# # --- Gaps de meses hacia adelante
# ds[, gap1 := periodo1 - periodo0]   # distancia f0 → f1
# ds[, gap2 := periodo2 - periodo1]   # distancia f1 → f2
# 
# # --- Inicializo con CONTINUA donde hay horizonte suficiente
# ds[, clase_ternaria := NA_character_]
# ds[periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA"]
# 
# # --- BAJA+1: no hay futuro o regreso tardío (> tol)
# ds[
#   periodo0 < periodo_ultimo &
#   ( is.na(periodo1) | (!is.na(periodo1) & (gap1 - 1L) > tol) ),
#   clase_ternaria := "BAJA+1"
# ]
# 
# # --- BAJA+2: tiene f1 consecutivo y luego no hay futuro o regreso tardío (> tol)
# ds[
#   periodo0 < periodo_anteultimo &
#   (periodo1 == periodo0 + 1L) &
#   ( is.na(periodo2) | (!is.na(periodo2) & (gap2 - 1L) > tol) ),
#   clase_ternaria := "BAJA+2"
# ]
# 
# # --- Devuelvo al orden original y pego resultado al dataset
# setorder(ds, pos)
# DT[, clase_ternaria := ds$clase_ternaria]
# 
# DT[, .(Clientes_unicos = uniqueN(numero_de_cliente)), by=.(foto_mes, clase_ternaria)][order(foto_mes, clase_ternaria)]


### Eliminacion de Features

Completar a gusto LUEGO de realizar un analisis exploratorio de datos.
<br> No necesariamente en esta Segunda Competencia conviele eliminar los mismos campos que en la Primera ...

### prestamos personales

In [23]:
# Identificar columnas a eliminar
columnas_eliminar <- grep("prestamos_personales", names(dataset), value = TRUE)

dataset[, (columnas_eliminar) := NULL]
ncol(dataset)

[1] 153

### tmobileapp

In [ ]:
## TMOBILEAPP 202010
#detectado cambio de significado desde 202010 inclusive. qué hacer.

#dataset[, tmobile_app := ifelse(foto_mes < 202010, NA, tmobile_app)]


### var_tirar

In [ ]:
# vars_tirar <- c(
#   "cliente_vip",
#    "mrentabilidad",
#    "mactivos_margen",
#    "ccuenta_corriente",
#   "cdescubierto_preacordado",
#   "cprestamos_prendarios",
#   "mprestamos_prendarios",
#   "minversion2",
#   "mpayroll2",
#   "cpayroll2_trx",
#   "cpagodeservicios",
#   "mpagodeservicios",
#   "ccajeros_propios_descuentos",
#   "mcajeros_propios_descuentos",
#   "ctarjeta_visa_descuentos",
#   "mtarjeta_visa_descuentos",
#   "ctarjeta_master_descuentos",
#   "mtarjeta_master_descuentos",
#   "ccomisiones_mantenimiento",
#   "mcomisiones_mantenimiento",
#  "mcheques_emitidos_rechazados",
#  "tmobile_app",
#   "cmobile_app_trx",
#   "Master_delinquency",
#   "Master_Finiciomora",
#   "Master_msaldodolares",
#   "Master_mconsumosdolares",
#    "Master_madelantopesos",
#   "Master_madelantodolares",
#   "Master_mpagospesos",
#   "Master_mpagosdolares",
#     "Master_cadelantosefectivo",
#     "Visa_delinquency",
# "Visa_mfinanciacion_limite",
#  "Visa_Fvencimiento",
#   "Visa_Finiciomora",
#   "Visa_madelantopesos",
#   "Visa_madelantodolares",
#   "Visa_cadelantosefectivo",
#   "Visa_mpagominimo"
# )

# # Por seguridad: me quedo solo con las que realmente existen en el dataset
# vars_tirar_ok <- intersect(vars_tirar, names(dataset))

In [ ]:
# # Eliminar esas columnas
# dataset[, (vars_tirar_ok) := NULL]

# ncol(dataset)

### Variables_continuas

In [26]:
# variables continuas

variables_continuas <- c(
  # Variables de rentabilidad y comisiones
  "mrentabilidad", "mrentabilidad_annual", "mcomisiones",
  "mactivos_margen", "mpasivos_margen",

  # Cuentas y saldos principales
  "mcuenta_corriente", "mcaja_ahorro", "mcuentas_saldo",

  # Tarjetas - consumos
  "mtarjeta_visa_consumo", "mtarjeta_master_consumo",

  # Préstamos - montos
  "mprestamos_personales", "mprestamos_prendarios", "mprestamos_hipotecarios",

  # Inversiones - montos
  "mplazo_fijo_pesos", "mplazo_fijo_dolares",
  "minversion1_pesos", "minversion1_dolares", "minversion2",

  # Payroll
  "mpayroll", "mpayroll2",

  # Débitos automáticos
  "mcuenta_debitos_automaticos",

  # Servicios y pagos
  "mpagodeservicios", "mpagomiscuentas",

  # Comisiones
  "mcomisiones_mantenimiento", "mcomisiones_otras",

  # Forex
  "mforex_buy", "mforex_sell",

  # Transferencias
  "mtransferencias_recibidas", "mtransferencias_emitidas",

  # Extracciones y ATM
  "mextraccion_autoservicio", "matm", "matm_other",

  # Cheques
  "mcheques_depositados", "mcheques_emitidos",

  # Tarjetas Master y Visa - saldos y pagos
  "Master_msaldototal", "Master_msaldopesos", "Master_msaldodolares",
  "Master_mconsumospesos", "Master_mconsumosdolares",
  "Master_mlimitecompra", "Master_madelantopesos", "Master_madelantodolares",
  "Master_mpagado", "Master_mpagospesos", "Master_mpagosdolares",
  "Master_mconsumototal", "Master_mpagominimo",

  "Visa_msaldototal", "Visa_msaldopesos", "Visa_msaldodolares",
  "Visa_mconsumospesos", "Visa_mconsumosdolares",
  "Visa_mlimitecompra", "Visa_madelantopesos", "Visa_madelantodolares",
  "Visa_mpagado", "Visa_mpagospesos", "Visa_mpagosdolares",
  "Visa_mconsumototal", "Visa_mpagominimo",

  "mcuenta_corriente_adicional", "Master_mfinanciacion_limite", "Visa_mfinanciacion_limite",
"mcheques_emitidos_rechazados", "mcheques_depositados_rechazados", "mtarjeta_master_descuentos",
"mcaja_ahorro_adicional", "mtarjeta_visa_descuentos", "mttarjeta_master_debitos_automaticos",
"mcaja_ahorro_dolares", "mttarjeta_visa_debitos_automaticos", "mautoservicio", "mcajeros_propios_descuentos"
 )

cat("Variables continuas:", length(variables_continuas), "\n")
cat("Variables:\n")
print(variables_continuas)

# Verificar que todas las variables existen en el dataset
variables_existentes <- intersect(variables_continuas, names(dataset))
variables_faltantes <- setdiff(variables_continuas, names(dataset))

if(length(variables_faltantes) > 0) {
  cat("\n️ ADVERTENCIA: Variables no encontradas en dataset:\n")
  print(variables_faltantes)
}

variables_continuas <- variables_existentes
cat("\nVariables confirmadas en dataset:", length(variables_continuas), "\n")

Variables continuas: 73 
Variables:
 [1] "mrentabilidad"                       
 [2] "mrentabilidad_annual"                
 [3] "mcomisiones"                         
 [4] "mactivos_margen"                     
 [5] "mpasivos_margen"                     
 [6] "mcuenta_corriente"                   
 [7] "mcaja_ahorro"                        
 [8] "mcuentas_saldo"                      
 [9] "mtarjeta_visa_consumo"               
[10] "mtarjeta_master_consumo"             
[11] "mprestamos_personales"               
[12] "mprestamos_prendarios"               
[13] "mprestamos_hipotecarios"             
[14] "mplazo_fijo_pesos"                   
[15] "mplazo_fijo_dolares"                 
[16] "minversion1_pesos"                   
[17] "minversion1_dolares"                 
[18] "minversion2"                         
[19] "mpayroll"                            
[20] "mpayroll2"                           
[21] "mcuenta_debitos_automaticos"         
[22] "mpagodeservicios"                 

### variables top 40 
sin lags, deltas ni ranks

In [27]:
cols_top40 <- c(
  "ctrx_quarter",
  "ctrx_quarter_normalizado",
  "mcaja_ahorro",
  "cpayroll_trx",
  "mtarjeta_visa_consumo",
  "Visa_status",
  "mcuentas_saldo",
  "foto_mes",
  "ctarjeta_visa_transacciones",
  "cdescubierto_preacordado_tend6",
  "mpayroll",
  "Visa_status_tend6",
  "mpasivos_margen",
  "Visa_Finiciomora",
  "cdescubierto_preacordado",
  "mactivos_margen",
  "mcuenta_corriente",
  "mpayroll_sobre_edad",
  "ccomisiones_mantenimiento_tend6",
  "ctarjeta_visa",
  "cproductos",
  "mcuenta_corriente_tend6",
  "Visa_msaldototal",
  "Visa_mpagospesos",
  "Visa_msaldopesos",
  "mcomisiones_mantenimiento_tend6",
  "Visa_mpagominimo",
  "mcuentas_saldo_tend6",
  "cproductos_tend6",
  "mrentabilidad_annual",
  "ccaja_ahorro",
  "Master_status",
  "cliente_edad",
  "Master_fechaalta",
  "mttarjeta_visa_debitos_automaticos_tend6",
  "internet",
  "ctrx_quarter_tend6",
  "ctarjeta_visa_debitos_automaticos_tend6",
  "mrentabilidad",
  "Visa_delinquency"
)



### Data Quality

### NA si batch == 0

In [28]:
#EXPLORO

res_wide <- dataset[
  , lapply(.SD, function(v) {
      # condición estricta: sin NAs y todos exactamente 0
      if (!all(!is.na(v))) return(FALSE)
      all(v == 0)
    }),
  by = foto_mes
]

cero_por_batch <- melt(
  res_wide,
  id.vars = "foto_mes",
  variable.name = "variable",
  value.name = "todo_cero"
)[todo_cero == TRUE, .(variable, foto_mes)]

cero_por_batch


variable,foto_mes
<fct>,<int>
active_quarter,202006
mrentabilidad,201905
mrentabilidad,201910
mrentabilidad,202006
mrentabilidad_annual,201905
mrentabilidad_annual,201910
mrentabilidad_annual,202006
mcomisiones,201905
mcomisiones,201910


In [29]:
# APLICO
NA_of <- function(v){
  if (inherits(v, "integer64")) {
    if (requireNamespace("bit64", quietly = TRUE)) return(bit64::NA_integer64_)
    return(NA)  # fallback si no está bit64
  }
  if (is.integer(v))   return(NA_integer_)
  if (is.numeric(v))   return(NA_real_)
  if (is.logical(v))   return(NA)
  if (is.character(v)) return(NA_character_)
  # factor, Date, etc. → NA genérico
  NA
}

vars <- unique(cero_por_batch$variable)

for (var in vars) {
  meses <- cero_por_batch[variable == var, foto_mes]
  na_val <- NA_of(dataset[[var]])
  
  # comparación a 0 según tipo (para evitar coerciones raras)
  if (is.character(dataset[[var]])) {
    dataset[foto_mes %in% meses & get(var) == "0", (var) := na_val]
  } else {
    dataset[foto_mes %in% meses & get(var) == 0,  (var) := na_val]
  }
}


### monto NA si c == 0

In [ ]:
# Diccionario 1: (c -> m)  aplicar NA a m cuando c == 0/"0"

dict_c0_mNA <- list(

)

# Diccionario 2: (c -> m) aplicar NA a m cuando (c == 0/"0"  y  m == 0/"0")
dict_c0_m0_mNA <- list(
  c("ccaja_ahorro","mcaja_ahorro"),
  c("ccaja_ahorro","mcaja_ahorro_dolares"),
  c("ccajeros_propios_descuentos","mcajeros_propios_descuentos"),
  c("ccheques_depositados_rechazados","mcheques_depositados_rechazados"),
  c("ccheques_emitidos_rechazados","mcheques_emitidos_rechazados"),
  c("ccomisiones_mantenimiento","mcomisiones_mantenimiento"),
  c("ccuenta_corriente","mcuenta_corriente"),
  c("cextraccion_autoservicio","mextraccion_autoservicio"),
  c("cforex_buy","mforex_buy"),
  c("cforex_sell","mforex_sell"),
  c("cinversion1","minversion1_dolares"),
  c("cinversion1","minversion1_pesos"),
  c("cinversion2","minversion2"),
  c("cpagodeservicios","mpagodeservicios"),
  c("cpagomiscuentas","mpagomiscuentas"),
  c("cplazo_fijo","mplazo_fijo_dolares"),
  c("cplazo_fijo","mplazo_fijo_pesos"),
  c("cprestamos_hipotecarios","mprestamos_hipotecarios"),
  c("cprestamos_prendarios","mprestamos_prendarios"),
  c("ctarjeta_visa", "Visa_mlimitecompra"),
  c("ctarjeta_master", "Master_mlimitecompra"),
  c("ccheques_depositados","mcheques_depositados"),
  c("ccheques_emitidos","mcheques_emitidos")
)

# Diccionario 3: ([c1,c2] -> [m1,m2]) aplicar NA a ambos m cuando (c1 == 0/"0"  y  c2 == 0/"0")
dict_c0c0_mNAmNA <- list(
  list(c_cols = c("cpayroll_trx","cpayroll2_trx"),
       m_cols = c("mpayroll","mpayroll2"))
)

# Helpers
NA_of <- function(v){
  if (inherits(v, "integer64")) return(if (requireNamespace("bit64", quietly=TRUE)) bit64::NA_integer64_ else NA)
  if (is.integer(v))   return(NA_integer_)
  if (is.numeric(v))   return(NA_real_)
  if (is.logical(v))   return(NA)
  if (is.character(v)) return(NA_character_)
  NA
}

is_zero <- function(x){
  if (is.character(x) || is.factor(x)) x == "0" else x == 0
}

In [ ]:
# APLICO monto NA si c 0

if (length(dict_c0_mNA) > 0) {
  for (par in dict_c0_mNA) {
    c_col <- par[1]
    m_col <- par[2]
    
    dataset[
      is_zero(get(c_col)),
      (m_col) := NA_of(get(m_col))
    ]
  }
}

if (length(dict_c0_m0_mNA) > 0) {
  for (par in dict_c0_m0_mNA) {
    c_col <- par[1]
    m_col <- par[2]
    
    dataset[
      is_zero(get(c_col)) & is_zero(get(m_col)),
      (m_col) := NA_of(get(m_col))
    ]
  }
}

if (length(dict_c0c0_mNAmNA) > 0) {
  for (regla in dict_c0c0_mNAmNA) {
    c1 <- regla$c_cols[1]
    c2 <- regla$c_cols[2]
    m_cols <- regla$m_cols
    
    dataset[
      is_zero(get(c1)) & is_zero(get(c2)),
      (m_cols) := lapply(.SD, NA_of),
      .SDcols = m_cols
    ]
  }
}

### Data Drifting

### Inflación

In [ ]:
# inflación por IPC
ipc_var <- c(
  2.9, 3.8, 4.7, 3.4, 3.1, 2.7, 2.2, 4.0, 5.9, 3.3, 4.3, 3.7,
  2.3, 2.0, 3.3, 1.5, 1.5, 2.2, 1.9, 2.7, 2.8, 3.8, 3.2, 4.0,
  4.0, 3.6, 4.8, 4.1, 3.3, 3.2, 3.0, 2.5
  ,  3.5   # (202109)
)

# Genero la secuencia de foto_mes 201901..202108
seq_fm <- function(from_fm, to_fm){
  dseq <- seq(as.Date(paste0(from_fm,"01"), "%Y%m%d"),
              as.Date(paste0(to_fm,"01"), "%Y%m%d"), by="month")
  as.integer(format(dseq, "%Y%m"))
}
fm_seq <- seq_fm(201901L, 202109L)

stopifnot(length(ipc_var) == length(fm_seq))

ipc <- data.table(
  foto_mes = fm_seq,
  variacion_mensual = ipc_var
)

# Índice acumulado (base libre). Pongo 100 en el primer mes y acumulo.
ipc[, indice := 100 * cumprod(1 + variacion_mensual/100)]

# ================= Deflactar a una base elegida =================
base_mes <- 202109L            # último mes disponible
stopifnot(base_mes %in% ipc$foto_mes)

# Traigo el índice al dataset y calculo factor relativo a la base
setkey(ipc, foto_mes)
stopifnot("foto_mes" %in% names(dataset))
dataset <- ipc[dataset, on="foto_mes"]

indice_base <- ipc[foto_mes == base_mes, indice][1]
dataset[, defl_factor := indice / indice_base]  # >1 = precios más altos que la base

# Detecto columnas de montos (m* y Master_m*/Visa_m*)
montos_m_prefix <- grep("^m", names(dataset), value = TRUE)
montos_card     <- grep("^(Master|Visa)_m", names(dataset), value = TRUE)
montos_all <- unique(c(montos_m_prefix, montos_card))
montos_all <- montos_all[vapply(dataset[, ..montos_all], is.numeric, logical(1))]

# Deflactar IN-PLACE a pesos de 'base_mes'
for (cn in montos_all) {
  dataset[, (cn) := get(cn) / pmax(defl_factor, 1e-12)]
}

# (opcional) trazabilidad y limpieza
dataset[, ipc_base := base_mes]
dataset[, c("variacion_mensual","indice","defl_factor") := NULL]

cat(sprintf("IPC aplicado (base %d). Columnas deflactadas: %d\n", base_mes, length(montos_all)))

ncol(dataset)

IPC aplicado (base 202109). Columnas deflactadas: 72


[1] 154

### Rankings

In [ ]:
# cat("Variables continuas:", length(variables_continuas), "\n")

# # Rankings con cero fijo por (foto_mes):
# # positivos -> (0,1], negativos -> [-1,0), ceros -> 0, NA -> NA

# ranking_cero_fijo <- function(x) {
#   positivos <- x > 0 & !is.na(x)
#   negativos <- x < 0 & !is.na(x)
#   ceros     <- x == 0 & !is.na(x)
#   nas       <- is.na(x)

#   n_pos <- sum(positivos)
#   n_neg <- sum(negativos)

#   resultado <- numeric(length(x))
#   resultado[nas]   <- NA
#   resultado[ceros] <- 0

#   if (n_pos > 0) {
#     ranks_pos <- data.table::frank(x[positivos], ties.method = "average", na.last = "keep") # OPCION RANDOM
#     resultado[positivos] <- ranks_pos / n_pos                 # (0,1], 1 si hay un único positivo
#   }
#   if (n_neg > 0) {
#     ranks_neg <- data.table::frank(x[negativos], ties.method = "average", na.last = "keep")
#     resultado[negativos] <- -(n_neg - ranks_neg + 1) / n_neg  # [-1,0), -1 si hay un único negativo
#   }
#   resultado
# }




In [ ]:
# # APLICO RANKINGS 

# vars_ranking <- variables_continuas
  
# # Aplicar ranking por foto_mes
# for (var in vars_ranking) {
#   if (var %in% names(dataset)) {
#     col_rank <- paste0(var, "_rank")
#     dataset[, (col_rank) := ranking_cero_fijo(get(var)), by = foto_mes]
#     cat("Ranking creado:", col_rank, "\n")
#   }
# }

# # Validación de rangos
# vars_rank <- paste0(vars_ranking[vars_ranking %in% names(dataset)], "_rank")
# cat("\n=== VALIDACIÓN DE RANGOS ===\n")
# dataset[, lapply(.SD, function(x) {
#   c(min = round(min(x, na.rm = TRUE), 6), 
#     max = round(max(x, na.rm = TRUE), 6),
#     ceros = sum(x == 0, na.rm = TRUE))
# }), .SDcols = vars_rank]

# cat("\nRankings aplicados correctamente\n")

# variables_continuas_ranking <- grep("_rank$", names(dataset), value = TRUE)
# ncol(dataset)
# # print(variables_continuas_ranking)

### Feature Engineering Intra-Mes

Crear variables nuevas a partir de las existentes dentro del mismo registro, **sin** ir a buscar información histórica.
<br> El siguiente código es un mínimo ejemplo, agregar nuevos features a gusto

In [ ]:
# el mes 1,2, ..12 , podria servir para detectar estacionalidad
dataset[, kmes := foto_mes %% 100]

# creo un ctr_quarter que tenga en cuenta cuando los clientes hace 3 menos meses que estan 
#ya que seria injusto considerar las transacciones medidas en menor tiempo
dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]
dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5.0]
dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2.0]
dataset[cliente_antiguedad == 3, ctrx_quarter_normalizado := ctrx_quarter * 1.2]

dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

ncol(dataset)
Sys.time()

### Feature Engineering Historico

In [ ]:
if( !require("Rcpp")) install.packages("Rcpp", repos = "http://cran.us.r-project.org")
require("Rcpp")

In [ ]:
# se calculan para los 6 meses previos el minimo, maximo y
#  tendencia calculada con cuadrados minimos
# la formula de calculo de la tendencia puede verse en
#  https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
# para la maxíma velocidad esta funcion esta escrita en lenguaje C,
# y no en la porqueria de R o Python

cppFunction("NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde )
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 5*n );

  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;


    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) )
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h<libre; h++)
      {
        xsum  += x[h] ;
        ysum  += y[h] ;
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;

        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ;
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}")

In [ ]:
# calcula la tendencia de las variables cols de los ultimos 6 meses
# la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
# La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas <- function(
    dataset, cols, ventana = 6, tendencia = TRUE,
    minimo = TRUE, maximo = TRUE, promedio = TRUE,
    ratioavg = FALSE, ratiomax = FALSE) {
  gc(verbose= FALSE)
  # Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion <- ventana

  last <- nrow(dataset)

  # creo el vector_desde que indica cada ventana
  # de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids <- dataset[ , numero_de_cliente ]

  vector_desde <- seq(
    -ventana_regresion + 2,
    nrow(dataset) - ventana_regresion + 1
  )

  vector_desde[1:ventana_regresion] <- 1

  for (i in 2:last) {
    if (vector_ids[i - 1] != vector_ids[i]) {
      vector_desde[i] <- i
    }
  }
  for (i in 2:last) {
    if (vector_desde[i] < vector_desde[i - 1]) {
      vector_desde[i] <- vector_desde[i - 1]
    }
  }

  for (campo in cols) {
    nueva_col <- fhistC(dataset[, get(campo)], vector_desde)

    if (tendencia) {
      dataset[, paste0(campo, "_tend", ventana) :=
        nueva_col[(0 * last + 1):(1 * last)]]
    }

    if (minimo) {
      dataset[, paste0(campo, "_min", ventana) :=
        nueva_col[(1 * last + 1):(2 * last)]]
    }

    if (maximo) {
      dataset[, paste0(campo, "_max", ventana) :=
        nueva_col[(2 * last + 1):(3 * last)]]
    }

    if (promedio) {
      dataset[, paste0(campo, "_avg", ventana) :=
        nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratioavg) {
      dataset[, paste0(campo, "_ratioavg", ventana) :=
        get(campo) / nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratiomax) {
      dataset[, paste0(campo, "_ratiomax", ventana) :=
        get(campo) / nueva_col[(2 * last + 1):(3 * last)]]
    }
  }
}

### lags deltas

no laguear las continuas cuando haya rankings

In [ ]:
# Creacion de LAGs (excluí monetarias originales)
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy(setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria"
   # , variables_continuas
   # ,vars_rank
    )
))

# lags de orden 1
dataset[,
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 2
dataset[,
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}

ncol(dataset)
Sys.time()

### tendencias 6

In [ ]:
# parametros de Feature Engineering Historico de Tendencias
PARAM$FE_hist$Tendencias$run <- TRUE
PARAM$FE_hist$Tendencias$ventana <- 6
PARAM$FE_hist$Tendencias$tendencia <- TRUE
PARAM$FE_hist$Tendencias$minimo <- FALSE
PARAM$FE_hist$Tendencias$maximo <- FALSE
PARAM$FE_hist$Tendencias$promedio <- FALSE
PARAM$FE_hist$Tendencias$ratioavg <- TRUE
PARAM$FE_hist$Tendencias$ratiomax <- FALSE

# aqui se agregan las tendencias de los ultimos 6 meses

#cols_iniciales <- intersect(cols_iniciales, colnames(dataset))
# cols_top40 <- intersect(cols_top40, colnames(dataset))
cols_lagueables <- intersect(cols_lagueables, colnames(dataset))

setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    TendenciaYmuchomas(dataset,
    cols = cols_lagueables,
    #cols = cols_iniciales,
    ventana = PARAM$FE_hist$Tendencias$ventana, # 6 meses de historia
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}

ncol(dataset)
Sys.time()

### tendencias 3

In [ ]:

# VENTANA 3 y más
# parametros de Feature Engineering Historico de Tendencias
PARAM$FE_hist$Tendencias$run <- TRUE
PARAM$FE_hist$Tendencias$ventana <- 3
PARAM$FE_hist$Tendencias$tendencia <- TRUE
PARAM$FE_hist$Tendencias$minimo <- FALSE
PARAM$FE_hist$Tendencias$maximo <- FALSE
PARAM$FE_hist$Tendencias$promedio <- FALSE
PARAM$FE_hist$Tendencias$ratioavg <- TRUE
PARAM$FE_hist$Tendencias$ratiomax <- FALSE

# aqui se agregan las tendencias de los ultimos 3 meses

#cols_iniciales <- intersect(cols_iniciales, colnames(dataset))
# cols_top40 <- intersect(cols_top40, colnames(dataset))
cols_lagueables <- intersect(cols_lagueables, colnames(dataset))

setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    TendenciaYmuchomas(dataset,
    cols = cols_lagueables,
    # cols = cols_iniciales,
    ventana = PARAM$FE_hist$Tendencias$ventana, # 6 meses de historia
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}
ncol(dataset)
Sys.time()

### aguinaldo flag 

clientes que tuvieron payroll , tienen al menos 1 ctrxpayroll en mes aguinaldo y su mpayroll de mes aguinaldo es al menos 40% mayor que el anterior (me quedan clientes con ctrxpayroll positivo sin asignar a la columna). Por lo tanto lo que identifico no son empleados, sino aproximo aguinaldos.

In [ ]:
# dataset[, recibio_aguinaldo := 0L]

# # Marco 1 solo donde se cumplen TODAS las condiciones
# dataset[
#   substr(foto_mes, 5, 6) %in% c("01", "06", "07", "12") &  # meses candidatos
#     cpayroll_trx_lag2 > 0 &
#     cpayroll_trx_lag1 > 0 &
#     cpayroll_trx      > 0 &
#     !is.na(mpayroll)        & !is.na(mpayroll_lag1)       & !is.na(mpayroll_lag2) &
#     mpayroll        > 0     & mpayroll_lag1        > 0    & mpayroll_lag2        > 0 &
#     mpayroll > 1.4 * mpayroll_lag1,                        # +40% vs mes anterior
#   recibio_aguinaldo := 1L
# ]
# ncol(dataset)

### intrames sc

In [ ]:
vars_sc <- c(
"ctrx_quarter_normalizado",
#"mcaja_ahorro_rank",
"mcaja_ahorro",
"ctrx_quarter",
#"mtarjeta_visa_consumo_rank",
#"mcuenta_corriente_rank",
"mtarjeta_visa_consumo",
"mcuenta_corriente",
#"Visa_status",
"cpayroll_trx",
#"mcuentas_saldo_rank",
"mcuentas_saldo",
#"mpayroll_rank",
#"mactivos_margen_rank",
"ctarjeta_visa_transacciones",
#"mpayroll_sobre_edad_rank",
"mpayroll_sobre_edad",
#"mpasivos_margen_rank",
#"Visa_mpagominimo_rank",
"ccomisiones_mantenimiento_tend6",
"cproductos_ratioavg6"
)

# Ratios
for(variable_num in vars_sc){
  for(variable_den in vars_sc){
    if(variable_num != variable_den){
      nuevo_nombre <- paste0(variable_num, "_sobre_", variable_den)
    
      dataset[, (nuevo_nombre) := get(variable_num) / fifelse(get(variable_den)==0, NA_real_, get(variable_den))]
    }
  }
}

# productos
for(factor_1 in vars_sc){
  for(factor_2 in vars_sc){
    if(factor_1 != factor_2){
      nuevo_nombre <- paste0(factor_1, "_por_", factor_2)
      dataset[, (nuevo_nombre) := get(factor_1) * get(factor_2)]
    }
  }
}

### Feature Engineering RF

In [ ]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

In [ ]:
# AgregaVarRandomForest <- function() {

#   cat( "inicio AgregaVarRandomForest()\n")
#   gc(verbose= FALSE)
#   dataset[, clase01 := 0L ]
#   dataset[ clase_ternaria %in% c( "BAJA+2", "BAJA+1"),
#       clase01 := 1L ]

#   campos_buenos <- setdiff(
#     colnames(dataset),
#     c( "clase_ternaria", "clase01")
#   )

#   dataset[, entrenamiento :=
#     as.integer( foto_mes %in% PARAM$FE_rf$train$training )]

#   dtrain <- lgb.Dataset(
#     data = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
#     label = dataset[entrenamiento == TRUE, clase01],
#     free_raw_data = FALSE
#   )

#   modelo <- lgb.train(
#      data = dtrain,
#      param = PARAM$FE_rf$lgb_param,
#      verbose = -100
#   )

#   cat( "Fin construccion RandomForest\n" )
#   # grabo el modelo, achivo .model
#   lgb.save(modelo, file="modelo.model" )

#   qarbolitos <- copy(PARAM$FE_rf$lgb_param$num_iterations)

#   periodos <- dataset[ , unique( foto_mes ) ]

#   for( periodo in  periodos )
#   {
#     cat( "periodo = ", periodo, "\n" )
#     datamatrix <- data.matrix(dataset[ foto_mes== periodo, campos_buenos, with = FALSE])

#     cat( "Inicio prediccion\n" )
#     prediccion <- predict(
#         modelo,
#         datamatrix,
#         type = "leaf"
#     )
#     cat( "Fin prediccion\n" )

#     for( arbolito in 1:qarbolitos )
#     {
#        cat( arbolito, " " )
#        hojas_arbol <- unique(prediccion[ , arbolito])

#        for (pos in 1:length(hojas_arbol)) {
#          # el numero de nodo de la hoja, estan salteados
#          nodo_id <- hojas_arbol[pos]
#          dataset[ foto_mes== periodo, paste0(
#             "rf_", sprintf("%03d", arbolito),
#              "_", sprintf("%03d", nodo_id)
#           ) :=  as.integer( nodo_id == prediccion[ , arbolito]) ]

#        }

#        rm( hojas_arbol )
#     }
#     cat( "\n" )

#     rm( prediccion )
#     rm( datamatrix )
#     gc(verbose= FALSE)
#   }

#   gc(verbose= FALSE)

#   # borro clase01 , no debe ensuciar el dataset
#   dataset[ , clase01 := NULL ]

# }

In [ ]:
# # Parametros de Feature Engineering  a partir de hojas de Random Forest

# # Estos CUATRO parametros son los que se deben modificar
# PARAM$FE_rf$arbolitos= 20
# PARAM$FE_rf$hojas_por_arbol= 16
# PARAM$FE_rf$datos_por_hoja= 100
# PARAM$FE_rf$mtry_ratio= 0.2

# # Estos son quasi fijos
# PARAM$FE_rf$train$training <- c( 202101, 202102, 202103)

# # Estos TAMBIEN son quasi fijos
# PARAM$FE_rf$lgb_param <-list(
#     # parametros que se pueden cambiar
#     num_iterations = PARAM$FE_rf$arbolitos,
#     num_leaves  = PARAM$FE_rf$hojas_por_arbol,
#     min_data_in_leaf = PARAM$FE_rf$datos_por_hoja,
#     feature_fraction_bynode  = PARAM$FE_rf$mtry_ratio,

#     # para que LightGBM emule Random Forest
#     boosting = "rf",
#     bagging_fraction = ( 1.0 - 1.0/exp(1.0) ),
#     bagging_freq = 1.0,
#     feature_fraction = 1.0,

#     # genericos de LightGBM
#     max_bin = 31L,
#     objective = "binary",
#     first_metric_only = TRUE,
#     boost_from_average = TRUE,
#     feature_pre_filter = FALSE,
#     force_row_wise = TRUE,
#     verbosity = -100,
#     max_depth = -1L,
#     min_gain_to_split = 0.0,
#     min_sum_hessian_in_leaf = 0.001,
#     lambda_l1 = 0.0,
#     lambda_l2 = 0.0,

#     pos_bagging_fraction = 1.0,
#     neg_bagging_fraction = 1.0,
#     is_unbalance = FALSE,
#     scale_pos_weight = 1.0,

#     drop_rate = 0.1,
#     max_drop = 50,
#     skip_drop = 0.5,

#     extra_trees = FALSE
#   )

In [ ]:
# # Feature Engineering agregando variables de Random Forest
# #  aqui es donde se hace el trabajo
# AgregaVarRandomForest()

# Sys.time()
# sum(startsWith(names(dataset), "rf_"))

In [ ]:
dim(dataset)
#colnames(dataset)

# Produccion

In [ ]:
# utilizo  zLightGBM  la nueva libreria
if( !require("zlightgbm") ) install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/zlightgbm_4.6.0.99.tar.gz", repos= NULL, type= "source")
require("zlightgbm")
Sys.time()

### Final Training Strategy

In [ ]:
PARAM$train_final$future <- c(202109)  # POR APO
PARAM$train_final$false_future <- c(202109)

PARAM$train_final$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908, 201909, 201910, 201911, 201912
, 202001, 202002, 202003
, 202004, 202005, 202006, 
202007, 202008, 202009, 202010
, 202011, 202012
, 202101, 202102, 202103, 202104, 202105
, 202106, 202107
)

PARAM$train_final$undersampling <- 0.1

In [ ]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$training]

#### Registros cambio las proporciones de POS/NEG

In [ ]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [ ]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

## Canaritos

In [ ]:


cat("Columnas ANTES:", ncol(dataset_train_final), "\n")

PARAM$qcanaritos <- 5L
cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)
for(i in seq(PARAM$qcanaritos)) {
  dataset_train_final[, paste0("canarito_", i) := runif(filas)]
}
cols_canaritos <- setdiff(colnames(dataset_train_final), cols0)
setcolorder(dataset_train_final, c(cols_canaritos, cols0))


cat("Canaritos creados:", length(cols_canaritos), "\n")
cat("Columnas DESPUÉS:", ncol(dataset_train_final), "\n")


### Final Models

In [ ]:
# Semillerio Final APO
PARAM$train_final$APO <- 1
PARAM$train_final$ksemillerio  <- 30

PARAM$train_final$cortes <- c(10500, 11000, 11500, 12000, 12500)

In [ ]:
library(primes)
primos <- generate_primes(min = 100000, max = 1000000)
# =================== hasta acá

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
PARAM$train_final$semillas <- sample(primos)[seq( PARAM$train_final$ksemillerio*PARAM$train_final$APO )] # APO
PARAM$train_final$semillas

In [ ]:
# # dejo los datos en formato LightGBM

# # APO CAMPOS BUENOS SE AGREGA ACÁ SINO ESTABA EN MODELADO
# campos_buenos <- setdiff(colnames(dataset_train_final), c("clase_ternaria","clase01","training"))


# dtrain_final <- lgb.Dataset(
#   data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
#   label= dataset_train_final[training == 1L, clase01],
#   free_raw_data= FALSE
# )

# cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
# Sys.time()

#### train pesos

In [ ]:
# # PESOS ASIGNADOS MANUALMENTE

# dataset_train_final[, peso_mes := 1.0]  # valor por defecto

# dataset_train_final[foto_mes %in% c(
#   201905
#   , 201910
#   , 202006
# ), peso_mes := 0.2]

# dataset_train_final[foto_mes %in% c(
#   201901, 201902, 201903, 201904, 201906, 
#   201907, 201908, 201909, 201911, 201912
# ), peso_mes := 0.5]

# dataset_train_final[foto_mes %in% c(
#   202001, 202002, 202003, 202004, 202005
  
#   , 202007, 202008, 202009
  
# ), peso_mes := 0.8]

# dataset_train_final[foto_mes %in% c(
# 202010, 202011, 202012
#  , 202101, 202102, 202103, 202104, 202105
#  #, 202106, 202107
# ), peso_mes := 1.0]


In [ ]:
# PESOS ASIGNADOS DECAIMIENTO EXPONENCIAL

# Obtener los foto_mes que efectivamente están en el training
meses_training <- sort( unique(dataset_train_final[training == 1L, foto_mes]) )

# Asigno un índice 0,1,2,... en orden temporal a esos meses
map_mes <- data.table(
  foto_mes = meses_training,
  idx_mes  = seq_along(meses_training) - 1L   # 0 para el más viejo, max para el más nuevo
)

# junto este índice a cada fila según su foto_mes
dataset_train_final[
  map_mes,
  on = "foto_mes",
  idx_mes := i.idx_mes
]

# calculo los "pasos hacia atrás" desde el mes más reciente de training
max_idx <- dataset_train_final[training == 1L, max(idx_mes)]

# Base del decaimiento 
base_decay <- 0.95

# Para las filas de training: peso_mes = 0.95^(pasos_atras)
dataset_train_final[training == 1L,
  pasos_atras := max_idx - idx_mes
]

dataset_train_final[training == 1L,
  peso_mes := base_decay ^ pasos_atras
]

dataset_train_final[, c("idx_mes", "pasos_atras") := NULL]

# chequeo que los pesos por mes tengan sentido
dataset_train_final[training == 1L,
  .(peso_promedio = mean(peso_mes)),
  by = foto_mes
][order(foto_mes)][]


In [ ]:
# DTRAIN_FINAL PARA ASIGNAR PESOS

#CAMPOS BUENOS CUANDO NO USO PESOS EN MESES DE TRAIN
# campos_buenos <- setdiff(colnames(dataset_train_final), c("clase_ternaria","clase01","training"))

# CAMPOS BUENOS CUANDO USO PESOS EN meses
campos_buenos <- setdiff(colnames(dataset_train_final),
                         c("clase_ternaria","clase01","training","peso_mes"))
                         
dtrain_final <- lgb.Dataset(
  data   = data.matrix(dataset_train_final[training == 1L, campos_buenos, with = FALSE]),
  label  = dataset_train_final[training == 1L, clase01],
  weight = dataset_train_final[training == 1L, peso_mes],
  free_raw_data = FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

In [ ]:
PARAM$lgbm <- list(
  boosting          = "gbdt",
  objective         = "binary",
  metric            = "custom",
  first_metric_only = FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise    = TRUE,
  verbosity         = -100,
  seed              = PARAM$semilla_primigenia,
  max_bin           = 31L,
  min_data_in_leaf  = 40L,  #default era 20
  num_iterations    = 9999L,  #default era 9999L
  num_leaves        = 9999L,
  learning_rate     = 1.0,
  feature_fraction  = 0.50,
  canaritos         = PARAM$qcanaritos,
  gradient_bound    = 0.1   #default era 0.1    
)


In [ ]:
# Para compatibilidad con el bucle de semillerio de workflowtest:
PARAM$train_final$param_mejores <- PARAM$lgbm

## modelitos

In [ ]:
# FINAL TRAIN

dir.create("modelitos", showWarnings = FALSE)

param_completo <- copy(PARAM$train_final$param_mejores)

for (sem in PARAM$train_final$semillas) {
  arch_modelo <- sprintf("./modelitos/mod_%s_%s.txt", PARAM$experimento, sem)

  if (!file.exists(arch_modelo)) {
    param_completo$seed <- sem

    modelito <- lgb.train(
      data  = dtrain_final,
      param = param_completo
    )

    lgb.save(modelito, filename = arch_modelo)
    rm(modelito); gc()
    cat(" Modelo guardado seed", sem, "\n")
  } else {
    cat("Ya existía, salto seed", sem, "\n")
  }
}
cat("\nListo. Si no había pendientes, no se recalculó nada.\n")


# PRODUCCION 

### Scoring prod

Se hace el predict() del modelo en los datos del futuro

In [ ]:
# SCORING REAL

PARAM$train_final$future <- c(202109)

dfuture <- dataset[foto_mes %in% PARAM$train_final$future]

cat("Columnas ANTES:", ncol(dfuture), "\n")

######################

cols_fut_0 <- copy(colnames(dfuture))

filas_fut <- nrow(dfuture)
for(i in seq(PARAM$qcanaritos)) {
  dfuture[, paste0("canarito_", i) := runif(filas_fut)] 
}

cols_canaritos_fut <- setdiff(colnames(dfuture), cols_fut_0)
setcolorder(dfuture, c(cols_canaritos_fut, cols_fut_0))

cat("Canaritos creados:", length(cols_canaritos_fut), "\n")
cat("Columnas DESPUÉS:", ncol(dfuture), "\n")

#########################

mfuture <- data.matrix(dfuture[, campos_buenos, with = FALSE])

vpred_acum <- rep(0.0, nrow(dfuture))
qacumulados <- 0L

for(sem in PARAM$train_final$semillas) {
  arch_modelo <- sprintf("./modelitos/mod_%s_%s.txt", PARAM$experimento, sem)
  modelo_final <- lgb.load(arch_modelo)
  vpred_individual <- predict(modelo_final, mfuture)

  tb_pred_seed <- dfuture[, .(numero_de_cliente, foto_mes)]
  tb_pred_seed[, prob := vpred_individual ]
  fwrite(tb_pred_seed,
         file = sprintf("prediccion_future_real_%s_%s.txt", PARAM$experimento, sem),
         sep  = "\t")

  vpred_acum <- vpred_acum + vpred_individual
  qacumulados <- qacumulados + 1L
}

vpred_acum <- vpred_acum / qacumulados

tb_prediccion <- dfuture[, .(numero_de_cliente, foto_mes)]
tb_prediccion[, prob := vpred_acum ]
fwrite(tb_prediccion,
       file = sprintf("prediccion_future_real_%s.txt", PARAM$experimento),
       sep  = "\t")

### Clasificacion prod 

In [ ]:
# # genero archivos con los "envios" mejores
# dir.create("kaggle", showWarnings = FALSE)

# PARAM$cortes <- seq(10000, 13000, by = 500)

# # ordeno por probabilidad descendente
# setorder(tb_prediccion, -prob)

# envios <- 11000
# tb_prediccion[, Predicted := 0L]           # seteo inicial a 0
# tb_prediccion[1:envios, Predicted := 1L]   # marco los primeros

# archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

# # grabo el archivo
# fwrite(
#   tb_prediccion[, .(numero_de_cliente, Predicted)],
#   file = archivo_kaggle,
#   sep = ","
# )


In [ ]:
# genero archivos con los "envios" mejores
dir.create("kaggle_real", showWarnings = FALSE)

PARAM$cortes <- seq(10000, 13000, by = 500)

# ordeno por probabilidad descendente (no reordeno más de una vez)
tb_ord <- copy(tb_prediccion)
setorder(tb_ord, -prob)

# genero un archivo por cada corte en PARAM$cortes
for (envios in PARAM$cortes) {
  tb_ord[, Predicted := 0L]                          # seteo inicial a 0
  tb_ord[1:min(envios, .N), Predicted := 1L]         # marco los primeros

  archivo_kaggle <- paste0("./kaggle_real/KA_sil", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_ord[, .(numero_de_cliente, Predicted)],
        file = archivo_kaggle,
        sep = ",",
        col.names = TRUE)

  cat("Generado:", basename(archivo_kaggle), "\n")
}


In [ ]:
Sys.time()

# YML

In [ ]:
cat("\n=== Guardando parámetros del experimento ===\n")
if (!requireNamespace("yaml", quietly = TRUE)) install.packages("yaml")

# Construir primero la versión "plana"
PARAM_export <- list(
  experimento  = PARAM$experimento,
  train_final  = list(
    future        = PARAM$train_final$future,
    training      = PARAM$train_final$training,
    undersampling = PARAM$train_final$undersampling,
    semillas      = PARAM$train_final$semillas,
    param_mejores = PARAM$train_final$param_mejores
  ),
  cortes         = if ("cortes" %in% names(PARAM)) PARAM$cortes else NULL,
  qcanaritos     = if ("qcanaritos" %in% names(PARAM)) PARAM$qcanaritos else NULL,
  campos_buenos  = if (exists("campos_buenos")) campos_buenos else NULL
)

nombre_archivo_param <- sprintf("PARAM_plano_exp_%s.yml", PARAM$experimento)
yaml::write_yaml(PARAM_export, file = nombre_archivo_param)
cat("Archivo de parámetros guardado exitosamente en:", nombre_archivo_param, "\n")


In [ ]:
# R
# ----------------------------------------------------------------------
# ** PAUSA PARA ASEGURAR EL AUTOGUARDADO EN LA VM **
# ----------------------------------------------------------------------

print("Workflow finalizado. Pausando la ejecución por 60 segundos...")

# Pausa la ejecución del kernel R. Durante este tiempo, el servidor 
# de JupyterLab (en la VM) debería ejecutar el autoguardado.
Sys.sleep(60) 

print("Pausa completada. El autoguardado debería haber guardado el .ipynb.")